In [6]:
#!/usr/bin/python
import os
from  osgeo import ogr, osr
import pickle
from affine_transformation import *

def GetOutfeature(outLayer, outLayerDefn,geom,inFeature): #copy infeature to outLayer, and sets geom into feature.
    outFeature = ogr.Feature(outLayerDefn)
    outFeature.SetGeometry(geom)
    for i in range(0, outLayerDefn.GetFieldCount()):
        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))
    return outFeature

def createOutLayer(inlayer,outputShapefile, outlayername,outprojref,outgeomtype=ogr.wkbMultiPolygon):
    #create a data-set.
    outLayer=outLayerDefn=None
    if os.path.exists(outputShapefile):
        driver.DeleteDataSource(outputShapefile)
    outDataSet = driver.CreateDataSource(outputShapefile)
    
    # create output layer, copy fields from inlayer fielddefn
    outLayer = outDataSet.CreateLayer(outlayername,outprojref,geom_type=outgeomtype)
    
    inLayerDefn = inlayer.GetLayerDefn()
    
    for i in range(0, inLayerDefn.GetFieldCount()):
        fieldDefn = inLayerDefn.GetFieldDefn(i)
        outLayer.CreateField(fieldDefn)
    
    outLayerDefn = outLayer.GetLayerDefn()
    return outDataSet,outLayer, outLayerDefn

    
epsgdic = {'nad83':4269,'wgs84':4326,'pseudoutm':3857,'worldmercater':3395}
home =r'/home/naresh-1/workspace/machinelrn/data/gis/'
srcshpfile = home + r'out/prj_usa.shp'
outputShapefile = home+ r'out/prj.trans.int.usa.shp'
tv = pickle.load(open(home+r'out/translation_vector.p','rb'))

driver = ogr.GetDriverByName('ESRI Shapefile')
shp = driver.Open(srcshpfile)
inlayer = shp.GetLayer()
outDataSet,outLayer, outLayerDefn = createOutLayer(inlayer,outputShapefile,"testlayer",inlayer.GetSpatialRef(),outgeomtype=ogr.wkbMultiPolygon)

#read the features one by one.
inFeature = inlayer.GetNextFeature()
while inFeature:
    geom= inFeature.GetGeometryRef()
    geom=translate(geom, xoff=-tv['xoff'], yoff=-tv['yoff'], zoff=0.0,enlarge=10000) #xoff val is -ve.#in affine_transformation.
    #geom=translate(geom, xoff=-tv['xoff'], yoff=-tv['yoff'], zoff=0.0) #xoff val is -ve.
    # get the cover attribute for the input feature
    outFeature = GetOutfeature(outLayer, outLayerDefn,geom,inFeature)
    outLayer.CreateFeature(outFeature)
    
    inFeature=outFeature=geom=None
    inFeature = inlayer.GetNextFeature()
print("extents inlayer/outlayer"),inlayer.GetExtent(), outLayer.GetExtent()
shp = None
outDataSet = None
outLayer=None
print("completed cordinate transform.")

object type 'POLYGON'
extents inlayer/outlayer (-13885233.414708788, -7452828.220063132, 2845159.5575605105, 6307879.040906803) (61522749281.0, 125846801227.0, 183417302973.0, 218044497806.0)
completed cordinate transform.
